In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
%cd Z:\PPMI_Data\Excels\Only_Male\GeneCpG\Fatigue
data = pd.read_csv('FatigueCateg_Methylome_Male.csv')
data.head(1)

Z:\PPMI_Data\Excels\Only_Male\GeneCpG\Fatigue


,Unnamed: 0,Sentrix,PATNO,HYS,Gender,APPRDX,MDSP_Fatigue,cg17960051,cg06562372,cg04099454,...,cg15500865,cg10329418,cg18241308,cg04350520,cg04685170,cg20985067,cg08461772,cg25879118,cg07510163,ch.13.693633F
0,0,200991620021_R01C01,3001,2,1,1,1,0.019716,0.024434,0.895146,...,0.105174,0.262518,0.784256,0.886425,0.080717,0.814912,0.331552,0.916387,0.841837,0.036872


In [3]:
X = data.drop(['Unnamed: 0','Sentrix','PATNO','HYS','Gender','MDSP_Fatigue','APPRDX'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 

In [4]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [5]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.7096774193548387

In [6]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=2)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100


BorutaPy finished running.

Iteration: 	10 / 100
Confirmed: 	1
Tentative: 	0
Rejected: 	31211


BorutaPy(estimator=RandomForestClassifier(n_estimators=14,
                                          random_state=RandomState(MT19937) at 0x226F7774E40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x226F7774E40, verbose=1)

In [7]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  1


In [8]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [9]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(3)

,CpG,Ranking
0,APPRDX,1
3859,cg18473117,2
28277,cg17001689,3


In [10]:
Top50.to_csv ('Fatigue_BorutaMale_Top50.csv', index = False)
selected_rf_features.to_csv ('Fatigue_BorutaMale_All.csv', index = False)

In [ ]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['APPRDX'] = data['APPRDX']
df_subset.to_csv ('Fatigue_Boruta_Top50_Data_Male.csv', index = False)
df_subset.head(2)

#Identify a string in dataframe
substring = 'cg15328505'
selected_rf_features[selected_rf_features.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
